In [3]:
import pandas as pd
import numpy as np

In [4]:
# Dateien einlesen
df_de = pd.read_excel("../data/export_questions_de_20250310.xlsx")
df_fr = pd.read_excel("../data/export_questions_fr_20250310.xlsx")
df_it = pd.read_excel("../data/export_questions_it_20250310.xlsx")
df_alt = pd.read_excel("../data/sv-Fragen 2011-2021.xlsx")

# Spalten anzeigen (zur Kontrolle)
print("Deutsch:", df_de.columns.tolist())
print("Französisch:", df_fr.columns.tolist())
print("Italienisch:", df_it.columns.tolist())
print("Alt:", df_alt.columns.tolist())


Deutsch: ['ID_Wahl', 'Wahl_tag', 'Datum', 'Question_tag_ID', 'Question_tag', 'Frage_ID', 'Frage_Text', 'Sprache', 'Frage_Typ', 'Bereich_ID', 'Bereich']
Französisch: ['ID_Wahl', 'Wahl_tag', 'Datum', 'Question_tag_ID', 'Question_tag', 'Frage_ID', 'Frage_Text', 'Sprache', 'Frage_Typ', 'Bereich_ID', 'Bereich']
Italienisch: ['ID_Wahl', 'Wahl_tag', 'Datum', 'Question_tag_ID', 'Question_tag', 'Frage_ID', 'Frage_Text', 'Sprache', 'Frage_Typ', 'Bereich_ID', 'Bereich']
Alt: ['ID_Politikbereich', 'Politikbereich', 'ID_Wahl', 'Kanton', 'Jahr', 'ID_Frage', 'Frage', 'ID_sv-Kategorie', 'ID_sv-Group', 'Fragetyp']


In [5]:
# Altdatei anpassen: Spalten umbenennen & auf relevante Spalten reduzieren
df_alt_angepasst = df_alt.rename(columns={
    "ID_Wahl": "ID_Wahl",
    "Jahr": "Datum",
    "ID_Frage": "Frage_ID",
    "Frage": "Frage_Text",
    "Fragetyp": "Frage_Typ",
    "ID_Politikbereich": "Bereich_ID",
    "Politikbereich": "Bereich"
})

# Auswahl der Spalten wie in den neuen Dateien
spalten = ['ID_Wahl', 'Datum', 'Frage_ID', 'Frage_Text', 'Frage_Typ', 'Bereich_ID', 'Bereich']
df_alt_angepasst = df_alt_angepasst[spalten]
df_de = df_de[spalten]
df_fr = df_fr[spalten]
df_it = df_it[spalten]

In [6]:
df_de.head()

df_alt_angepasst.head()

,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich
0,76.0,2015,1169.0,Soll die Schweiz innerhalb der nächsten vier J...,Standard-4,1.0,Aussenpolitik
1,76.0,2015,1134.0,Befürworten Sie das bestehende Personenfreizüg...,Standard-4,1.0,Aussenpolitik
2,76.0,2015,1168.0,Soll die Schweiz mit den USA über ein Freihand...,Standard-4,1.0,Aussenpolitik
3,76.0,2015,1167.0,Die Schweiz verfolgt seit einigen Jahren eine ...,Standard-4,1.0,Aussenpolitik
4,76.0,2015,1171.0,Bildung,Budget-5,2.0,Bildung


In [7]:
# Alle DataFrames zusammenführen
df_gesamt = pd.concat([df_de, df_fr, df_it, df_alt_angepasst], ignore_index=True)
print(f"Gesamtanzahl der Fragen: {len(df_gesamt)}")

Gesamtanzahl der Fragen: 8997


In [8]:
# Eindeutige ID pro Zeile
df_gesamt["ID_gesamt"] = ["Q" + str(i).zfill(5) for i in range(len(df_gesamt))]

In [9]:
df_gesamt.head()

,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich,ID_gesamt
0,2.0,2019-10-20 00:00:00,5.0,"Wie beurteilen Sie diese Aussage: ""Wer sich ni...",options7,4734.0,Werthaltungen,Q00000
1,2.0,2019-10-20 00:00:00,8.0,"Wie beurteilen Sie diese Aussage: ""Die Bestraf...",options7,4734.0,Werthaltungen,Q00001
2,2.0,2019-10-20 00:00:00,11.0,"Wie beurteilen Sie diese Aussage: ""Für ein Kin...",options7,4734.0,Werthaltungen,Q00002
3,2.0,2019-10-20 00:00:00,14.0,Soll der Bund Ausländer/-innen bei der Integra...,options4,4231.0,Migration & Integration,Q00003
4,2.0,2019-10-20 00:00:00,17.0,Soll der Konsum von Cannabis legalisiert werden?,options4,4352.0,Gesellschaft & Ethik,Q00004


In [10]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def erkenne_sprache(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unbekannt"


In [11]:
df_gesamt["Sprache"] = df_gesamt["Frage_Text"].apply(erkenne_sprache)

In [34]:
print(df_gesamt["Sprache"].value_counts())

Sprache
de    7459
fr    1224
it     314
Name: count, dtype: int64


In [13]:
# Alle Zeilen, deren Sprache NICHT de, fr oder it ist
df_andere_sprachen = df_gesamt[~df_gesamt["Sprache"].isin(["de", "fr", "it"])]

# Anzahl anzeigen
print(f"{len(df_andere_sprachen)} Fragen wurden nicht als Deutsch, Französisch oder Italienisch erkannt.")

30 Fragen wurden nicht als Deutsch, Französisch oder Italienisch erkannt.


In [14]:
df_andere_sprachen.head()

,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich,ID_gesamt,Sprache
536,22.0,2020-09-27 00:00:00,1887.0,Soll der Kanton Solaranlagen an Gebäudefassade...,options4,5075.0,"Umwelt, Verkehr & Energie",Q00536,sv
3013,1106.0,2024-04-21 00:00:00,32478.0,Sollte der Kanton Gemeindefusionen stärker för...,options4,11498.0,Politisches System & Digitalisierung,Q03013,sv
3919,29.0,2020-10-25 00:00:00,2419.0,Devrait-on mettre en place davantage de zones ...,options4,5397.0,Verkehr & Infrastruktur,Q03919,nl
4025,62.0,2021-03-07 00:00:00,3463.0,Devrait-on mettre en place davantage de zones ...,options4,5404.0,Verkehr & Infrastruktur,Q04025,nl
4586,123.0,2015,1909.0,Kultur,Budget-5,8.0,"Kultur, Sport & Medien",Q04586,et


In [24]:
# Liste der IDs aus df_andere_sprachen
ids_andere_sprachen = df_andere_sprachen["ID_gesamt"].tolist()

# Anzahl und Beispielausgabe
print(f"{len(ids_andere_sprachen)} IDs gefunden.")
print(ids_andere_sprachen)


30 IDs gefunden.
['Q00536', 'Q03013', 'Q03919', 'Q04025', 'Q04586', 'Q04619', 'Q04646', 'Q04665', 'Q04830', 'Q05155', 'Q05203', 'Q05315', 'Q05407', 'Q05492', 'Q06399', 'Q06470', 'Q06695', 'Q06951', 'Q07097', 'Q07216', 'Q07235', 'Q07403', 'Q07451', 'Q07770', 'Q07932', 'Q08323', 'Q08550', 'Q08595', 'Q08596', 'Q08653']


In [35]:
df_gesamt.to_excel("df_gesamt_final.xlsx", index=False, engine="openpyxl")


In [36]:
df_de_final = df_gesamt[df_gesamt["Sprache"] == "de"].reset_index(drop=True)
df_fr_final = df_gesamt[df_gesamt["Sprache"] == "fr"].reset_index(drop=True)
df_it_final = df_gesamt[df_gesamt["Sprache"] == "it"].reset_index(drop=True)


In [37]:
df_de_final.to_excel("df_de_final.xlsx", index=False, engine="openpyxl")
df_fr_final.to_excel("df_fr_final.xlsx", index=False, engine="openpyxl")
df_it_final.to_excel("df_it_final.xlsx", index=False, engine="openpyxl")
